In [1]:

%pip install matplotlib
%pip install torch
%pip install umap-learn
%pip install tiktoken
%pip install openai
%pip install umap-pytorch


Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 23.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tensorflow as tf

print(tf.config.list_physical_devices())

2024-06-13 21:56:27.066518: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 21:56:45.745179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 21:56:45.745233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 21:56:48.103678: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 21:56:52.483148: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-06-13 21:57:29.676106: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-13 21:57:39.362908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-13 21:57:39.363187: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
import warnings

warnings.simplefilter("ignore")

import time

import torch

from umap_based_rag import UMAP_Based_RAG_Util

from som_based_rag import SOM_Based_RAG_Util

from wiki_datasource import WikiEventsDataSource

from openai_vector_encoder import OpenAIEmbeddingsVectorEncoder

from openai_qa_chatbot import OpenAIQuestionAnswerChatBot

Matplotlib is building the font cache; this may take a moment.


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
torch.cuda.is_available()


True

In [ ]:
openai_key = "Your key here :D"

openai_vector_encoder_id = "text-embedding-ada-002"

openai_encoded_vector_dimensions = 1536

openai_tokenizer_name = "cl100k_base" 

openai_model_name = "gpt-3.5-turbo-instruct"

vector_encoder = OpenAIEmbeddingsVectorEncoder( openai_encoded_vector_dimensions, openai_vector_encoder_id, openai_key )

In [ ]:
event_years_to_fetch = [ 2022, 2023, 2024 ]
data_source = WikiEventsDataSource( event_years_to_fetch  )

In [ ]:
umap_driven_rag_util = UMAP_Based_RAG_Util( 
                                            vector_encoder = vector_encoder,
                                            n_neighbors = 10,
                                            min_dist = 0.1,
                                            n_components = 3,
                                            k_clusters = 50,
                                            metric = 'cosine',
                                            topk_bmu_for_indexing = 10,
                                            device = device
                                        )

In [ ]:
som_driven_rag_util = SOM_Based_RAG_Util( 
                                            vector_encoder = vector_encoder,
                                            som_lattice_height = 20,
                                            som_lattice_width = 30,
                                            learning_rate = 0.3,
                                            topk_bmu_for_indexing = 5,
                                            device = device
                                        )

In [ ]:
umap_driven_rag_util.load_n_vectorize_data( data_source )

In [ ]:
som_driven_rag_util.load_n_vectorize_data(data_source)

In [ ]:
umap_driven_rag_util.vectors.shape
som_driven_rag_util.vectors.shape

In [ ]:
umap_train_t1 = time.perf_counter()
umap_driven_rag_util.train_n_index_data_vectors()
umap_training_time = time.perf_counter() - umap_train_t1
print(umap_training_time)

In [ ]:
som_train_t1 = time.perf_counter()
som_driven_rag_util.train_n_index_data_vectors()
som_training_time = time.perf_counter() - som_train_t1
print(som_training_time)

In [ ]:
umap_driven_rag_util.find_semantically_similar_data( "Who won the 2024 Sri Lankan Presidential Election?", sim_threshold = 0.7 )

In [ ]:
som_driven_rag_util.find_semantically_similar_data( "Who won the 2024 Sri Lankan Presidential Election?", sim_threshold = 0.7 )

In [ ]:
umap_openai_chatbot = OpenAIQuestionAnswerChatBot( 
                                                vector_db_util = umap_driven_rag_util,
                                                openai_tokenizer_name = openai_tokenizer_name,
                                                openai_model_name = openai_model_name,
                                                openai_key = openai_key,
                                                question_input_max_token_count = 100,
                                                context_trim_percent = 0.1,
                                                device = device
                                            )

In [ ]:
som_openai_chatbot = OpenAIQuestionAnswerChatBot( 
                                                vector_db_util = som_driven_rag_util,
                                                openai_tokenizer_name = openai_tokenizer_name,
                                                openai_model_name = openai_model_name,
                                                openai_key = openai_key,
                                                question_input_max_token_count = 100,
                                                context_trim_percent = 0.1,
                                                device = device
                                            )

In [ ]:
sample_questions = [    ["Who won the 2024 Sri Lankan Presidential Election?","Ranil Wickremesinghe"],
                        ["Who won the 2022 soccer world cup?", "Argentina"],
                        ["When did Sweden join NATO?", "2024"],
                        ["Who joined NATO in 2023?", "Finland"],
                        ["Who joined NATO in 2024?", "Sweden"],
                        ["Which is the 31st member of NATO?", "Finland"],
                        ["Which is the 32nd member of NATO?", "Sweden"],
                        ["Who won the Cricket World Cup in 2023?", "Australia"],
                        ["Who defeated India in Cricket World Cup final in 2023?", "Australia"],
                        ["Name the former prime minister of Japan that was assassinated in 2022?", "Shinzo Abe"],
                        ["When did Chandrayaan-3 land near the south pole of the Moon?", "July 14, 2023"],
                        ["Where did Chandrayaan-3 land on the Moon?", "south pole"],
                        ["Who acquired Twitter in 2022?", "Elon Musk"],
                        ["Who owns Twitter?", "Elon Musk"],
                        ["Who acquired Activision Blizzard in 2023?", "Microsoft"],
                        ["What country was invaded by Russia in 2022?", "Ukraine"],
                        ["What year did Russia invade Ukraine?", "2022"],
                        ["When was Katanji Brown Jackson sworn into the supreme court?"],
                        ["Who was the first black woman sworn on the supreme court?"]
                   ]

In [ ]:
for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question[0]}" )
    answer = umap_openai_chatbot.find_answer_to_question_without_context( sample_question[0] )
    print( f"Answer: {answer}" )

In [ ]:
sim_threshold = 0.55
total_time = 0

for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question[0]}" )
    t1 = time.perf_counter()
    answer = umap_openai_chatbot.find_answer_to_question( sample_question[0], sim_threshold = sim_threshold)
    t2 = time.perf_counter()
    print( f"Answer: {answer}" )
    total_time += (t2-t1)
print(f"Average search time per question{total_time/len(sample_questions)}")

In [ ]:
total_time = 0

for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question[0]}" )
    t1 = time.perf_counter()
    answer = umap_openai_chatbot.find_answer_to_question( sample_question[0], sim_threshold = sim_threshold)
    t2 = time.perf_counter()
    print( f"Answer: {answer}" )
    total_time += (t2-t1)
print(f"Average search time per question{total_time/len(sample_questions)}")

### 